In [84]:
import numpy as np
import math as m
nm = 1.0e-9

In [118]:
def Cylinder(centers,radius):
    
    text = "lc = 1e-9;\n"
    lineNames = ""
    text+="radius = %s;\n"%(radius)
    text+="Point(1) = { -30*radius, -30*radius, 0, lc};\n"
    text+="Point(2) = { -30*radius, 30*radius, 0, lc};\n"
    text+="Point(3) = { 30*radius, 30*radius, 0, lc};\n"
    text+="Point(4) = { 30*radius, -30*radius, 0, lc};\n"
    text+="Line(1) = {1,2};\n"
    text+="Line(2) = {2,3};\n"
    text+="Line(3) = {3,4};\n"
    text+="Line(4) = {4,1};\n"
    text+="Line Loop(1) = {1,2,3,4};\n"
    #text+="Plane Surface(1) = {1};\n"
    
    ptID=5 
    cirNum = 10
    for i,center in enumerate(centers):

        text+="Point(%s) = { %s, %s, 0, lc};\n"%(ptID,center[0],center[1])
        ptID+=1
        
        text+="Point(%s) = { %s, %s, 0, lc};\n"%(ptID,center[0]+radius,center[1])
        ptID+=1
        
        text+="Point(%s) = { %s, %s, 0, lc};\n"%(ptID,center[0],center[1]-radius)
        ptID+=1
        
        text+="Point(%s) = { %s, %s, 0, lc};\n"%(ptID,center[0]-radius,center[1])
        ptID+=1
        
        text+="Point(%s) = { %s, %s, 0, lc};\n"%(ptID,center[0],center[1]+radius)
        ptID+=1
                                
        text+="Circle(%s) = { %s, %s, %s};\n"%(cirNum,ptID-4,ptID-5,ptID-3)
        cirNum+=1
        
        text+="Circle(%s) = { %s, %s, %s};\n"%(cirNum,ptID-3,ptID-5,ptID-2)
        cirNum+=1
        
        text+="Circle(%s) = { %s, %s, %s};\n"%(cirNum,ptID-2,ptID-5,ptID-1)
        cirNum+=1
        
        text+="Circle(%s) = { %s, %s, %s};\n"%(cirNum,ptID-1,ptID-5,ptID-4)
        
        
        
        
        text+="Line Loop(%s) = {%s,%s,%s,%s};\n"%(i+5,cirNum-3,cirNum-2,cirNum-1,cirNum)
        text+="Plane Surface(%s) = {%s};\n\n"%(i+5,i+5)
        cirNum+=1
        
        if i==0:
            lineNames+= str(i+5)    
        else:
            lineNames+= "," + str(i+5)
    text+= "Plane Surface(1) = {1,%s};\n"%(lineNames)
    text+= "//Physical Surface(1) = {1};\n"
    text+= "//Physical Surface(2) = {%s};\n"%(lineNames)
    for i,center in enumerate(centers):
        text+="""out%s[]=Extrude{0,0,1e-8}{Surface{%s};};\n"""%(i+1,i+5)
        #text+="Volume(%s) = {out%s[1]};\n"%(i+5,i+1)

    #texts= text[:-2] #this is stupid ugly
    text+="\n\n\n\n\n\n\n\n\n\n\n"
    return text


In [119]:
def Build(num, radius,bR):
    centers = getCenters(num,radius,bR)
    text = Cylinder(centers,radius)
    print "post cylinder"
    fileName = "tester.geo"
    fileLines = []
    fileLines.append(text)
    with open(fileName, "w") as text_file:
                for fileLine in fileLines:
                        text_file.write(fileLine)


    import os
    mshName = fileName.replace(".geo",".msh")
    xmlName = fileName.replace(".geo",".xml")
    print "pre OS"
    line = "gmsh -3 %s"%fileName
    #os.system(line)
    #line="dolfin-convert %s %s"%(mshName,xmlName)
    #os.system(line)
    print "post OS"
    print xmlName
    return xmlName


In [120]:
def getCenters(num,radius,bR):
    numS = num**2
    i=0
    centers = [];

    if num%2!=0:
        while i <num:
            j=0
            x = radius*(-(3*num-4)/2+3*i)
            while j < num:
                
                y = radius*(-(3*num-4)/2+3*j)
                if (x**2)+(y**2)<bR**2:
                    this = False
                else:
                    centers.append((x,y))
                j+=1
            i+=1

    if num%2==0:
        while i <num:
            j=0
            x = radius*(-(3*num-10)/2+3*i)
            while j < num:
                y = radius*(-(3*num-4)/2+3*j)
                if (x**2)+(y**2)<bR**2:
                    this = False
                else:
                    centers.append((x,y))
                j+=1
            i+=1
    return centers



In [121]:
radius = 1e-9
bR = 9e-9

In [122]:
Build(18,radius,bR)
print "nope"

post cylinder
pre OS
post OS
tester.xml
nope


In [117]:
print "sos"

sos
